In [1]:
from pathlib import Path
from tqdm.notebook import tqdm
import shutil
import utils2p
import yaml
import numpy as np

from rye2p.thorimage.settings import FunctionalImagingSettings

# set upstairs or downstairs microscope
microscope = 'downstairs-2p'

prj = 'natural_mixtures'
# set project directory paths
if prj == 'natural_mixtures':
    PRJ_DATA_DIR = Path("/local/matrix/Remy-Data/projects/natural_mixtures")
    RAW_DATA_DIR = Path("/local/tensor-archive/Remy/natural_mixtures/raw_data")
    NAS_PROC_DIR = PRJ_DATA_DIR.joinpath('processed_data')
elif prj == 'haoru_rotation':
    PRJ_DATA_DIR = Path("/local/tensor/Haoru/rotation_project")
    RAW_DATA_DIR = Path("/local/tensor/Haoru/rotation_project/raw_data")
    NAS_PROC_DIR = Path("/local/tensor/Haoru/rotation_project/processed_data")

In [2]:
folder = Path("/local/matrix/Remy-Data/projects/natural_mixtures/processed_data/2022-02-11")

meta_files = sorted(list(folder.rglob("kiwi_ea_eb_only/Experiment.xml")))
meta_files

[PosixPath('/local/matrix/Remy-Data/projects/natural_mixtures/processed_data/2022-02-11/1/kiwi_ea_eb_only/Experiment.xml'),
 PosixPath('/local/matrix/Remy-Data/projects/natural_mixtures/processed_data/2022-02-11/2/kiwi_ea_eb_only/Experiment.xml'),
 PosixPath('/local/matrix/Remy-Data/projects/natural_mixtures/processed_data/2022-02-11/3/kiwi_ea_eb_only/Experiment.xml')]

In [3]:
for meta_file in meta_files:
    thorimage_settings = FunctionalImagingSettings.from_xml(meta_file)

    # Save ThorImage settings to proc_mov_dir
    thorimage_settings.to_yaml(meta_file.with_name('thorimage_settings.yaml'))

In [4]:
ts_file = Path("/local/tensor/Remy/natural_mixtures/processed_data/2022-02-11/1/kiwi_ea_eb_only/timestamps.npy")
timestamps = np.load(ts_file, allow_pickle=True).item()

thorimage_settings = FunctionalImagingSettings.from_yaml(ts_file.with_name('thorimage_settings.yaml'))


In [5]:
thorimage_settings = thorimage_settings.to_dict()

In [6]:
for k, v in timestamps.items():
    print(f"{k}: {v.size}")

display(timestamps['scope_ict'])
display(timestamps['scope_fct'])


frame_times: 101421
stack_times: 1690
olf_ict: 57
olf_fct: 58
scope_ict: 1
scope_fct: 1


array([17.1])

array([17.1])

In [25]:
scope_fct = timestamps['scope_fct']
scope_ict = timestamps['scope_ict']

raw_frame_times = timestamps['frame_times']

In [26]:
from rye2p.fix_frame_times import frames_by_scope_pulse

scope_padding = 0.0

# Group the frames by scope trigger
scope_fct_padded = scope_fct + scope_padding
frame_times_by_pulse = frames_by_scope_pulse(raw_frame_times,
                                             scope_ict,
                                             scope_fct_padded
                                             )

if thorimage_settings['fast_z']:
    expected_n_timepoints_by_scope_pulse = np.array(
            [np.floor_divide(scope_frames.size, thorimage_settings['steps_per_frame'])
             for scope_frames in frame_times_by_pulse]
            )

    expected_n_frames_by_scope_pulse = (expected_n_timepoints_by_scope_pulse
                                        * thorimage_settings['steps_per_frame'])

    expected_n_timepoints = np.sum(expected_n_timepoints_by_scope_pulse)
    expected_n_frames = np.sum(expected_n_frames_by_scope_pulse)

    correct_n_frames = (expected_n_frames == thorimage_settings['n_frames'])
    correct_n_timepoints = (expected_n_timepoints == thorimage_settings['n_timepoints'])
    # %%
    print(
            f"\nCorrect # of frames: {correct_n_frames}"
            f"\nCorrect # of timepoints: {correct_n_timepoints}"

            f"\n\nExpected from filtering:"
            f"\n\tn_frames: {expected_n_frames}"
            f"\n\tn_timepoints: {expected_n_timepoints}"

            f"\n\nThorImage:"
            f"\n\tn_frames: {thorimage_settings['n_frames']}"
            f"\n\tn_timepoints: {thorimage_settings['n_timepoints']}\n\n"
            )


Correct # of frames: False
Correct # of timepoints: False

Expected from filtering:
	n_frames: 0
	n_timepoints: 0

ThorImage:
	n_frames: 101400
	n_timepoints: 5070


